In [2]:
import io
import json
import os
import sys
import pandas as pd
from bs4 import BeautifulSoup as bs

import requests
from requests.sessions import Session

import numpy as np
from scipy import stats
import datetime
import time
import statistics as st
import concurrent
from threading import Thread,local
import asyncio
import aiohttp
import tqdm
import nest_asyncio
import re

nest_asyncio.apply()

from sqlalchemy import create_engine,ForeignKey
from sqlalchemy.orm import declarative_base,relationship
from sqlalchemy import Column, Integer, String, Boolean, BIGINT, TEXT, INT, FLOAT, BOOLEAN
from sqlalchemy.dialects.sqlite import DATETIME
from sqlalchemy.orm import Session
Base = declarative_base()

engine = create_engine("sqlite:///test.sqlite", echo=False)

In [33]:
headers = {
    'Accept-Encoding': 'gzip, deflate, sdch',
    'Accept-Language': 'en-US,en;q=0.8',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
    'Cache-Control': 'max-age=0',
    'Connection': 'keep-alive',
}

class DataError(ValueError): pass
class PMUError(Exception): pass

class Partants_scrapper():
    def __init__(self, course, training=True):
        self.course = course
        self.courseId = course["id"]
        self.date = course["date"].strftime("%Y-%m-%d")
        self.idHippo = course["idHippo"]
        self.numCourse = course["numCourse"]
        self.numReunion = course["numReunion"]
        self.heure = course["heureCourse"]
        self.has_tracking = course["hasTracking"]
        self.r_tab_partant = None
        self.r_tab_arrivee = None
        self.r_partant_pmu = None
        
        self.distance = course["distance"]
        self.categorie = course["categorie"].split(" ")[1]
        
        self.training = training
        self.classement = None
        self.gains = None
        
        #try:
        loop = asyncio.get_event_loop()
        programme = loop.run_until_complete(asyncio.gather(self._info_tableau_partant()))

        df = pd.DataFrame(programme[0])
        df["rid"] = df["id"]
        df.drop('id', inplace=True, axis=1)
        if 'tauxReclamation' in list(df):
            df.drop('tauxReclamation', inplace=True, axis=1)

        if 'commentaireApresCourse_texte' in list(df):
            df.drop('commentaireApresCourse_texte', inplace=True, axis=1)   
        if 'commentaireApresCourse_source' in list(df):
            df.drop('commentaireApresCourse_source', inplace=True, axis=1)
        if 'handicapPoids' in list(df):
            df.drop('handicapPoids', inplace=True, axis=1) 
        if 'nomPereMere' in list(df):
            df.drop('nomPereMere', inplace=True, axis=1)
        self.info_partants = df.to_dict('records')

            
    def _reduc_to_sec(self,reduc):
        minutes = int(reduc[0])
        secondes = int(reduc[1:3])
        dixieme = int(reduc[3])
        return (60  * minutes + secondes + dixieme/10)*1000
        
        
    async def _request_tableau_partants(self, session):
        async with session.get(f"https://www.letrot.com/stats/fiche-course/{self.date}/{self.idHippo}/{self.numCourse}/partants/tableau", headers=headers) as response:
             r = await response.text()
        soup = bs(r, "html.parser")
        headers_table = soup.find("table", {"id": "result_table"}).find("thead").find("tr").find_all("th")
        table = soup.find("table", {"id": "result_table"}).find("tbody")
        rows = table.find_all("tr")
        self.r_tab_partant = rows, headers_table
    
    async def _request_tableau_arrive(self, session):
        async with session.get(f"https://www.letrot.com/stats/fiche-course/{self.date}/{self.idHippo}/{self.numCourse}/resultats/arrivee-definitive", headers=headers) as response:
             r = await response.text()
        soup = bs(r, "html.parser")
        headers_table = soup.find("table", {"id": "result_table"}).find("thead").find("tr").find_all("th")
        table = soup.find("table", {"id": "result_table"}).find("tbody")
        rows = table.find_all("tr")
        
        classement = {row.select("td")[1].text : row.select("td")[0].find("span", {"class": "bold"}).text for row in rows}
        gains = {row.select("td")[1].text : row.find("div", {"class": "gains"}).text.replace(" ", "").replace("€", "") for row in rows}
        self.classement = classement
        self.gains = gains
        self.r_tab_arrivee = rows,classement
        
    async def _request_tableau_tracking(self, session):
        async with session.get(f"https://www.letrot.com/stats/fiche-course/{self.date}/{self.idHippo}/{self.numCourse}/tracking", headers=headers) as response:
             r = await response.text()
        soup = bs(r, "html.parser")
        headers_table = soup.find("table", {"id": "result_table"}).find("thead").find("tr").find_all("th")
        table = soup.find("table", {"id": "result_table"}).find("tbody")
        rows = table.find_all("tr")
        
        tracking = []
        for r in rows:
            cheval = {}
            cheval["num_tracking"] = int(r.find("td", {"title": "Numéro"}).text)
            cheval["distPremArriv"] = int(r.find("td", {"title": "Distance par rapport au 1er à l'arrivée en mètres"}).span.text)
            cheval["reduc2000m"] = self._reduc_to_sec(r.find("td", {"title": "Réduction km aux 2 000 m"}).span.text)
            cheval["class2000m"] = int(r.find("td", {"title": "Classement aux 2 000 m"}).span.text)
            
            cheval["reduc1500m"] = self._reduc_to_sec(r.find("td", {"title": "Réduction km aux 1 500 m"}).span.text)
            cheval["class1500m"] = int(r.find("td", {"title": "Classement aux 1 500 m"}).span.text)
            
            cheval["reduc1000m"] = self._reduc_to_sec(r.find("td", {"title": "Réduction km aux 1 000 m"}).span.text)
            cheval["class1000m"] = int(r.find("td", {"title": "Classement aux 1 000 m"}).span.text)   
            
            cheval["reduc500m"] = self._reduc_to_sec(r.find("td", {"title": "Réduction km aux 500 m"}).span.text)
            cheval["class500m"] = int(r.find("td", {"title": "Classement aux 500 m"}).span.text)
            
            cheval["reducLast1000m"] = self._reduc_to_sec(r.find("td", {"title": "Réduction km derniers 1000 m"}).span.text)
            cheval["reducLast500m"] = self._reduc_to_sec(r.find("td", {"title": "Réduction km derniers 500 m"}).span.text)
            
            tracking.append(cheval)
            
        tracking.sort(key=lambda x: x["num_tracking"])
        
        self.tracking = tracking
    
    async def _request_partant_pmu(self, session):
        date_pmu = "".join(self.date.split("-")[::-1]) 
        async with session.get(f"https://online.turfinfo.api.pmu.fr/rest/client/65/programme/{date_pmu}/R{self.numReunion}/C{self.numCourse}/participants", headers=headers) as response:
             participants_pmu = await response.json()
        try:
            pmu_jsoned = participants_pmu["participants"]
            participants = pd.json_normalize(pmu_jsoned, sep="_").to_dict(orient="records")
            participants_with_id = [dict(item, **{"id": self.courseId, "numReunion": self.numReunion}) for item in participants]  
            self.r_partant_pmu = participants_with_id
        except:
            raise PMUError("Erreur API PMU")
            
        
        
    async def _info_tableau_partant(self):
        chevaux = []
        
        tasks = []
        async with aiohttp.ClientSession() as session:
            tasks.append(asyncio.ensure_future(self._request_tableau_partants(session)))
            tasks.append(asyncio.ensure_future(self._request_tableau_arrive(session)))
            tasks.append(asyncio.ensure_future(self._request_partant_pmu(session)))
#             tasks.append(asyncio.ensure_future(self.get_info_couple(session)))
#             tasks.append(asyncio.ensure_future(self.get_info_cheval_hippo(session)))
            if self.has_tracking:
                tasks.append(asyncio.ensure_future(self._request_tableau_tracking(session)))
            res = await asyncio.gather(*tasks, return_exceptions=True)
            
        for r in res:
            if type(r) == PMUError:
                raise PMUError("Erreur PMU")
       
        try:
            tableau_partants, headers_table = self.r_tab_partant
            tableau_arrivee,classement = self.r_tab_arrivee
            tableau_pmu = self.r_partant_pmu
        except:
            raise DataError("Erreur donnée")
        
        chevaux.extend(tableau_pmu)
        
        for i,row in enumerate(tableau_partants):
                num = row.select("td")[0].find("span", {"class": "bold"}).text
                col = row.select("td")
                cheval = {}
                cheval["num"] = num
                cheval["nom"] = col[1].text
                
                if self.has_tracking and num.isnumeric() and len(self.tracking) == len(tableau_partants):
                    cheval.update(self.tracking[i])
                
                cheval["numCoursePMU"] = f"R{self.numReunion}C{self.numCourse}"

                if self.training:
                    if num == "NP":
                        cheval["classement"] = "NP"
                    else:
                        cheval["classement"] = classement[num]
                      
                    if num != "NP":
                        cheval["gainCourse"] = int(self.gains[num])
                    else:
                        cheval["gainCourse"] = 0
                    
                cheval["id"] = self.courseId
                cheval["date"] = self.date
                cheval["url"] = col[1].find("a").get("href")

                cheval["heureCourse"] = self.heure
                cheval["fer"] = int(col[3].text) if col[3].text else 0
                cheval["firstTimeFer"] = 1 if col[3].find("div", {"class", "fer-first-time"}) else 0
                cheval["sex"] = 0 if col[4].text == "M" else 1
                cheval["age"] = int(col[5].text)
                cheval["dist"] = int(col[6].text.replace(" ", "").replace("\n", ""))
                cheval["driver"] = col[7].find("a").get("href")
                cheval["trainer"] = col[8].find("a").get("href")
                
                cheval["driver_id"] = cheval["driver"].split("/")[-3]
                cheval["trainer_id"] = cheval["trainer"].split("/")[-3]

                if "Avis" in headers_table[9].text:
                    cheval["avisTrainer"] = int(col[9].get("data-order"))
                    avis = col.pop(9)
                    col.insert(-1, avis)
                else:
                    cheval["avisTrainer"] = 2
                
                music = list(filter(lambda x: "a" in x, col[9].text.replace("D", "0").replace("Ret", "0").replace("T", "0").split()))

                try:
                    cheval["recordAbs"] = list(map(int, col[10].text.replace(col[10].span.text, "").replace("\'", '"').split('"')))
                    cheval["recordAbs"] = cheval["recordAbs"][0] * 10 * 60 + cheval["recordAbs"][1] * 10 + cheval["recordAbs"][2]
                except:
                    cheval["recordAbs"] = None

                cheval["gain"] = int(col[11].find("div", class_="gains").text.replace(" ", "")[:-1])
                
                chevaux[i].update(cheval)
        return chevaux
    
    
    async def get_tracking(self,session, url):
        async with session.get(url.replace("dernieres-performances", "tracking"), headers=headers) as response:
             r = await response.text()
        soup = bs(r, "html.parser")
        headers_table = soup.find("table", {"id": "result_table"}).find("thead").find("tr").find_all("th")
        table = soup.find("table", {"id": "result_table"}).find("tbody")
        rows = table.find_all("tr")
        
        info_tracking = {}
        
        distance_au_premier_arrivee = []
        accélération_500m = []
        gain_classement_500m = []
        for row in rows:
            dist_prem = int(row.find_all("td")[2].span.text)
            if dist_prem < 9999:
                distance_au_premier_arrivee.append(dist_prem)
            
            pre_fin = int(row.find_all("td")[17].span.text)
            fin = int(row.find_all("td")[18].span.text)
            if pre_fin < 2000 and fin < 2000:                                            
                accélération_500m.append(pre_fin - fin)
                

            try:
                class_500m = int(row.find_all("td")[16].span.text)
                class_final = int(row.find_all("td")[1].find("span", {"class": "bold"}).text)
                if class_500m -  class_final < 10:
                    gain_classement_500m.append(class_500m -  class_final)
            except:
                gain_classement_500m.append(0)
            
        info_tracking["mean_dist_arrivee"] = np.mean(distance_au_premier_arrivee) if len(distance_au_premier_arrivee) > 0 else np.nan
        info_tracking["acceleration_500m"] = np.mean(accélération_500m) if len(accélération_500m) > 0 else np.nan
        info_tracking["gain_classement_fin"] = np.mean(gain_classement_500m) if len(gain_classement_500m) > 0 else np.nan
        return info_tracking

In [37]:
courses_df = pd.read_csv("../data/coursestoday.csv", index_col=0)
courses_df["distance"] = courses_df["distance"].str.replace(" ", "").astype(int)
courses_df["date"] = courses_df["date"] + " " + courses_df["heureCourse"]
courses_df["date"] = pd.to_datetime(courses_df["date"])
courses_df.head()

,date,id,numReunion,hippodrome,idHippo,heureCourse,discipline,numCourse,prix,allocation,distance,categorie,typePiste,conditionsEngagement,hasTracking,hasVideoHeat,statut,classement,linkPrix,replay
0,2022-12-22 17:05:00,2022122214752,5,CABOURG,1475,17:05,Attelé,2,PRIX DES CINERAIRES,22 000,2750,Course F,NaN,"Pour chevaux entiers et hongres de 5 ans, ayan...",True,False,16,10 - 12 - 7 - 13 - 9,https://www.letrot.com/stats/fiche-course/2022...,"<a href=""https://www.letrot.com/fr/replay-cour..."
1,2022-12-22 17:38:00,2022122214753,5,CABOURG,1475,17:38,Attelé,3,PRIX DES MUFLIERS,29 000,2750,Course E,NaN,"Pour chevaux entiers et hongres de 6, 7 et 8 a...",True,False,16,1 - 9 - 6 - 3 - 13,https://www.letrot.com/stats/fiche-course/2022...,"<a href=""https://www.letrot.com/fr/replay-cour..."
2,2022-12-22 18:15:00,2022122214754,5,CABOURG,1475,18:15,Attelé,4,PRIX DES PELARGONIUM,21 000,2750,Course E,NaN,"Pour poulains entiers et hongres de 3 ans, n'a...",True,False,16,14 - 3 - 7 - 12 - 15,https://www.letrot.com/stats/fiche-course/2022...,"<a href=""https://www.letrot.com/fr/replay-cour..."
3,2022-12-22 18:45:00,2022122214755,5,CABOURG,1475,18:45,Attelé,5,PRIX DES FRAXINELLES,20 000,2750,Course D,NaN,"Pour 2 ans, n'ayant pas gagné 7.800.<br />Sont...",True,False,16,7 - 12 - 11 - 8 - 10,https://www.letrot.com/stats/fiche-course/2022...,"<a href=""https://www.letrot.com/fr/replay-cour..."
4,2022-12-22 19:15:00,2022122214756,5,CABOURG,1475,19:15,Attelé,6,PRIX DES ARISTOLOCHES,19 000,2750,Course F,NaN,"Pour poulains entiers et hongres de 4 ans, n'a...",True,False,16,7 - 1 - 9 - 11 - 13,https://www.letrot.com/stats/fiche-course/2022...,"<a href=""https://www.letrot.com/fr/replay-cour..."


In [19]:
testcourses = pd.read_sql("SELECT * FROM courses", con=engine)
testcourses["date"] = pd.to_datetime(testcourses["date"])

In [42]:
info_2 = []
t = time.time()
counter = 0

def gen_rows(df):
    for row in df.itertuples(index=False):
        yield row._asdict()
# ids = pd.read_csv("data/final_data_2122.csv").id.unique()
# c_filtered = c.loc[~c.id.isin(ids)]

for row in gen_rows(courses_df):
    res = Partants_scrapper(row)
    if isinstance(res.info_partants, list):
        try:
            with Session(engine) as session:
                partants = [Partant(**i) for i in res.info_partants]
                courses = [Course(**row, partants=partants)]
                session.add_all(courses)
                session.commit()
        except Exception as e:
            print(e)
            continue
#         info_2.extend(res.info_partants)
#         pd.DataFrame(res.info_partants).to_csv("data/final_data_2122.csv",mode="a", header=not os.path.isfile("data/final_data_2122.csv") ,index=False)
tps = time.time() - t
print(f"Fini en {int(tps//60)}min {int(tps%60)}s")

Fini en 0min 9s


In [35]:
df = pd.DataFrame(info_2)
df.rid.unique()

array([ 2022122475001,  2022122475003,  2022122475005,  2022122475006,
        2022122475008,  2022122475009, 20221224750010])

In [47]:
with Session(engine) as session:
    for i in session.query(Course).filter(Course.date >= "2022-12-24").all():
        print(i)

<Course(id='2022122475001', date='2022-12-24 12:08:00', numReunion='1', hippodrome='VINCENNES')>
<Course(id='2022122475003', date='2022-12-24 13:23:00', numReunion='1', hippodrome='VINCENNES')>
<Course(id='2022122475005', date='2022-12-24 14:33:00', numReunion='1', hippodrome='VINCENNES')>
<Course(id='2022122475006', date='2022-12-24 15:15:00', numReunion='1', hippodrome='VINCENNES')>
<Course(id='2022122475008', date='2022-12-24 16:31:00', numReunion='1', hippodrome='VINCENNES')>
<Course(id='2022122475009', date='2022-12-24 17:08:00', numReunion='1', hippodrome='VINCENNES')>
<Course(id='20221224750010', date='2022-12-24 17:48:00', numReunion='1', hippodrome='VINCENNES')>


In [581]:
list(test)

['id',
 'nom',
 'numPmu',
 'age',
 'sexe',
 'race',
 'statut',
 'placeCorde',
 'oeilleres',
 'proprietaire',
 'entraineur',
 'driver',
 'driverChange',
 'indicateurInedit',
 'musique',
 'nombreCourses',
 'nombreVictoires',
 'nombrePlaces',
 'nomPere',
 'nomMere',
 'ordreArrivee',
 'jumentPleine',
 'engagement',
 'supplement',
 'handicapDistance',
 'poidsConditionMonteChange',
 'tempsObtenu',
 'reductionKilometrique',
 'allure',
 'robe_code',
 'robe_libelleCourt',
 'robe_libelleLong',
 'dernierRapportDirect_typePari',
 'dernierRapportDirect_rapport',
 'dernierRapportDirect_typeRapport',
 'dernierRapportDirect_indicateurTendance',
 'dernierRapportDirect_nombreIndicateurTendance',
 'dernierRapportDirect_dateRapport',
 'dernierRapportDirect_permutation',
 'dernierRapportDirect_favoris',
 'dernierRapportDirect_numPmu1',
 'dernierRapportDirect_grossePrise',
 'deferre',
 'dernierRapportReference_typePari',
 'dernierRapportReference_rapport',
 'dernierRapportReference_typeRapport',
 'dernierRa

# SQL

In [4]:
from sqlalchemy import create_engine,ForeignKey
from sqlalchemy.orm import declarative_base,relationship
from sqlalchemy import Column, Integer, String, Boolean, BIGINT, TEXT, INT, FLOAT, BOOLEAN
from sqlalchemy.dialects.sqlite import DATETIME
from sqlalchemy.orm import Session
Base = declarative_base()

engine = create_engine("sqlite:///test.sqlite", echo=False)

In [552]:
# df.to_sql('partants', con=engine)

In [41]:
class Course(Base):
    __tablename__ = "courses"

#     id = Column(Integer, primary_key=True)
    id = Column(TEXT, primary_key=True)
    date = Column(DATETIME)
    numReunion = Column(INT)
    hippodrome = Column(TEXT)
    idHippo = Column(INT)
    heureCourse = Column(TEXT)
    discipline = Column(TEXT)
    numCourse = Column(INT)
    prix = Column(TEXT)
    allocation = Column(TEXT)
    distance = Column(INT)
    categorie = Column(TEXT)
    typePiste = Column(TEXT)
    conditionsEngagement = Column(TEXT)
    hasTracking = Column(BOOLEAN)
    hasVideoHeat = Column(BOOLEAN)
    statut = Column(INT)
    classement = Column(TEXT)
    linkPrix = Column(TEXT)
    replay = Column(TEXT)
    
    partants = relationship("Partant", order_by=Partant.numPmu, back_populates="course",
                            cascade="all, delete",
                            passive_deletes=True)
    
    def __repr__(self):
        return "<Course(id='%s', date='%s', numReunion='%s', hippodrome='%s')>" % (
            self.id,
            self.date,
            self.numReunion,
            self.hippodrome,
        )

In [40]:
class Partant(Base):
    __tablename__ = "partants"

    id = Column(Integer, primary_key=True)
    
    rid = Column(INT)
    nom = Column(TEXT)
    numPmu = Column(INT)
    age = Column(INT)
    sexe = Column(TEXT)
    race = Column(TEXT)
    statut = Column(TEXT)
    placeCorde = Column(FLOAT)
    oeilleres = Column(TEXT)
    proprietaire = Column(TEXT)
    entraineur = Column(TEXT)
    driver = Column(TEXT)
    driverChange = Column(BOOLEAN)
    indicateurInedit = Column(BOOLEAN)
    musique = Column(TEXT)
    nombreCourses = Column(INT)
    nombreVictoires = Column(INT)
    nombrePlaces = Column(INT)
    nomPere = Column(TEXT)
    nomMere = Column(TEXT)
    ordreArrivee = Column(FLOAT)
    jumentPleine = Column(BOOLEAN)
    engagement = Column(BOOLEAN)
    supplement = Column(INT)
    handicapDistance = Column(INT)
    poidsConditionMonteChange = Column(BOOLEAN)
    tempsObtenu = Column(FLOAT)
    reductionKilometrique = Column(FLOAT)
    allure = Column(TEXT)
    robe_code = Column(TEXT)
    robe_libelleCourt = Column(TEXT)
    robe_libelleLong = Column(TEXT)
    dernierRapportDirect_typePari = Column(TEXT)
    dernierRapportDirect_rapport = Column(FLOAT)
    dernierRapportDirect_typeRapport = Column(TEXT)
    dernierRapportDirect_indicateurTendance = Column(TEXT)
    dernierRapportDirect_nombreIndicateurTendance = Column(FLOAT)
    dernierRapportDirect_dateRapport = Column(FLOAT)
    dernierRapportDirect_permutation = Column(FLOAT)
    dernierRapportDirect_favoris = Column(TEXT)
    dernierRapportDirect_numPmu1 = Column(FLOAT)
    dernierRapportDirect_grossePrise = Column(TEXT)
    deferre = Column(TEXT)
    dernierRapportReference_typePari = Column(TEXT)
    dernierRapportReference_rapport = Column(FLOAT)
    dernierRapportReference_typeRapport = Column(TEXT)
    dernierRapportReference_indicateurTendance = Column(TEXT)
    dernierRapportReference_nombreIndicateurTendance = Column(FLOAT)
    dernierRapportReference_dateRapport = Column(FLOAT)
    dernierRapportReference_permutation = Column(FLOAT)
    dernierRapportReference_favoris = Column(TEXT)
    dernierRapportReference_numPmu1 = Column(FLOAT)
    dernierRapportReference_grossePrise = Column(TEXT)
    incident = Column(TEXT)
    numReunion = Column(INT)
    num = Column(TEXT)
    numCoursePMU = Column(TEXT)
    classement = Column(TEXT)
    date = Column(TEXT)
    url = Column(TEXT)
    heureCourse = Column(TEXT)
    fer = Column(INT)
    firstTimeFer = Column(INT)
    sex = Column(INT)
    dist = Column(INT)
    trainer = Column(TEXT)
    driver_id = Column(TEXT)
    trainer_id = Column(TEXT)
    avisTrainer = Column(INT)
    recordAbs = Column(FLOAT)
    gain = Column(INT)
    rid = Column(INT)
    ecurie = Column(TEXT)
    urlCasaque = Column(TEXT)
    nombrePlacesSecond = Column(FLOAT)
    nombrePlacesTroisieme = Column(FLOAT)
    eleveur = Column(TEXT)
    avisEntraineur = Column(TEXT)
    gainCourse = Column(INT)
    gainsParticipant_gainsCarriere = Column(FLOAT)
    gainsParticipant_gainsVictoires = Column(FLOAT)
    gainsParticipant_gainsPlace = Column(FLOAT)
    gainsParticipant_gainsAnneeEnCours = Column(FLOAT)
    gainsParticipant_gainsAnneePrecedente = Column(FLOAT)
    poidsConditionMonte = Column(FLOAT)
    num_tracking = Column(FLOAT)
    distPremArriv = Column(FLOAT)
    reduc2000m = Column(FLOAT)
    class2000m = Column(FLOAT)
    reduc1500m = Column(FLOAT)
    class1500m = Column(FLOAT)
    reduc1000m = Column(FLOAT)
    class1000m = Column(FLOAT)
    reduc500m = Column(FLOAT)
    class500m = Column(FLOAT)
    reducLast1000m = Column(FLOAT)
    reducLast500m = Column(FLOAT)
    
    race_id = Column(Integer, ForeignKey("courses.id", ondelete="CASCADE"))
    course = relationship("Course", back_populates="partants")

    def __repr__(self):
        return "<Partant(id='%s', numReunion='%s', classement='%s')>" % (
            self.course,
            self.numReunion,
            self.classement,
        )

In [632]:
Base.metadata.create_all(engine)

In [466]:
# with Session(engine) as session:
#     l_partant = df[df["rid"] == 2022091675001].to_dict("records")
# #     l_courses = courses_df[courses_df["id"] == 2022091675001].to_dict("records")
# #     print(l_courses)
#     partants = [Partant(**i) for i in l_partant]
#     # print(courses[courses["id"] == 2022091675001].to_dict("records")[0])
#     courses = [Course(**courses_df[courses_df["id"] == 2022091675001].to_dict("records")[0], partants=partants)]

#     session.add_all(courses)
#     session.commit()

2022-12-22 14:52:13,313 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-12-22 14:52:13,314 INFO sqlalchemy.engine.Engine INSERT INTO courses (id, date, "numReunion", hippodrome, "idHippo", "heureCourse", discipline, "numCourse", prix, allocation, distance, categorie, "typePiste", "conditionsEngagement", "hasTracking", "hasVideoHeat", statut, classement, "linkPrix", replay) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
2022-12-22 14:52:13,315 INFO sqlalchemy.engine.Engine [cached since 316s ago] (2022091675001, '2022-09-16 18:58:00.000000', 1, 'VINCENNES', 7500, '18:58', 'Attelé', 1, 'PRIX LUCRETIA', '39 000', 2100, 'Course D', nan, "Départ à l'autostart<br />Pour pouliches de 4 ans, n'ayant pas gagné 55.000.<br />", 1, 0, 16, '7 - 4 - 10 - 15 - 3', 'https://www.letrot.com/stats/fiche-course/2022-09-16/7500/1/resultats/arrivee-definitive', '<a href="https://www.letrot.com/fr/replay-courses/2022-09-16/7500/1" class="btn" target="_blank"><svg width="8px" height="

IntegrityError: (sqlite3.IntegrityError) UNIQUE constraint failed: courses.id
[SQL: INSERT INTO courses (id, date, "numReunion", hippodrome, "idHippo", "heureCourse", discipline, "numCourse", prix, allocation, distance, categorie, "typePiste", "conditionsEngagement", "hasTracking", "hasVideoHeat", statut, classement, "linkPrix", replay) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)]
[parameters: (2022091675001, '2022-09-16 18:58:00.000000', 1, 'VINCENNES', 7500, '18:58', 'Attelé', 1, 'PRIX LUCRETIA', '39 000', 2100, 'Course D', nan, "Départ à l'autostart<br />Pour pouliches de 4 ans, n'ayant pas gagné 55.000.<br />", 1, 0, 16, '7 - 4 - 10 - 15 - 3', 'https://www.letrot.com/stats/fiche-course/2022-09-16/7500/1/resultats/arrivee-definitive', '<a href="https://www.letrot.com/fr/replay-courses/2022-09-16/7500/1" class="btn" target="_blank"><svg width="8px" height="12px" viewBox="0 0 8 12" ve ... (213 characters truncated) ... rm="translate(4.000000, 6.000000) rotate(9

In [327]:
for i in df[df["id"] == 2022091675001].to_dict("records"):
    print(i)

{'nom': 'INES PICARDE', 'numPmu': 1, 'age': 4, 'sexe': 'FEMELLES', 'race': 'TROTTEUR FRANCAIS', 'statut': 'PARTANT', 'placeCorde': 1.0, 'oeilleres': 'SANS_OEILLERES', 'proprietaire': 'Ecurie J-F. SENET', 'entraineur': 'J.F. SENET', 'deferre': 'DEFERRE_ANTERIEURS_POSTERIEURS', 'driver': 'https://www.letrot.com/stats/fiche-homme/jf-senet/bGV5ZwMDeQ/jockey/dernieres-courses', 'driverChange': False, 'indicateurInedit': False, 'musique': '0a6aDa0a7a8a1a8a(21)4a', 'nombreCourses': 33, 'nombreVictoires': 1, 'nombrePlaces': 15, 'nombrePlacesSecond': 4, 'nombrePlacesTroisieme': 2, 'nomPere': "ALADIN D'ECAJEUL", 'nomMere': 'SWEET RAINBOW', 'ordreArrivee': 13.0, 'jumentPleine': False, 'engagement': False, 'supplement': 0, 'handicapDistance': 2100, 'poidsConditionMonteChange': False, 'tempsObtenu': 154490.0, 'reductionKilometrique': 73600.0, 'urlCasaque': 'https://www.pmu.fr/back-assets/hippique/casaques/16092022/R1/C1/P1.png', 'eleveur': 'E.A.R.L. Ecurie des TROTTEURS PICARDS', 'allure': 'TROT', 

In [328]:
courses[courses["id"] == 2022091675001].to_dict("records")

[{'date': Timestamp('2022-09-16 18:58:00'),
  'id': 2022091675001,
  'numReunion': 1,
  'hippodrome': 'VINCENNES',
  'idHippo': 7500,
  'heureCourse': '18:58',
  'discipline': 'Attelé',
  'numCourse': 1,
  'prix': 'PRIX LUCRETIA',
  'allocation': '39 000',
  'distance': 2100,
  'categorie': 'Course D',
  'typePiste': nan,
  'conditionsEngagement': "Départ à l'autostart<br />Pour pouliches de 4 ans, n'ayant pas gagné 55.000.<br />",
  'hasTracking': True,
  'hasVideoHeat': False,
  'statut': 16,
  'classement': '7 - 4 - 10 - 15 - 3',
  'linkPrix': 'https://www.letrot.com/stats/fiche-course/2022-09-16/7500/1/resultats/arrivee-definitive',
  'replay': '<a href="https://www.letrot.com/fr/replay-courses/2022-09-16/7500/1" class="btn" target="_blank"><svg width="8px" height="12px" viewBox="0 0 8 12" version="1.1" xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink"><g id="Page-1" stroke="none" stroke-width="1" fill="none" fill-rule="evenodd"><polygon id="triangle" fi

In [311]:
df[df["id"] == 2022091675001]

,nom,numPmu,age,sexe,race,statut,placeCorde,oeilleres,proprietaire,entraineur,...,sex,dist,trainer,driver_id,trainer_id,avisTrainer,recordAbs,gain,ecurie,incident
0,INES PICARDE,1,4,FEMELLES,TROTTEUR FRANCAIS,PARTANT,1.0,SANS_OEILLERES,Ecurie J-F. SENET,J.F. SENET,...,1,2100,https://www.letrot.com/stats/fiche-homme/jf-se...,bGV5ZwMDeQ,bGV5ZwMDeQ,2,725,48800,NaN,NaN
1,INES DE LA ROUVRE,2,4,FEMELLES,TROTTEUR FRANCAIS,PARTANT,2.0,SANS_OEILLERES,Ecurie L.M. DAVID,L.M. DAVID,...,1,2100,https://www.letrot.com/stats/fiche-homme/lm-da...,ZmRaYwYFdA,ZmRaYwYFdA,1,731,48680,NaN,NaN
2,ICE QUICK,3,4,FEMELLES,TROTTEUR FRANCAIS,PARTANT,3.0,SANS_OEILLERES,Ecurie QUICK STAR,P.J. CORDEAU,...,1,2100,https://www.letrot.com/stats/fiche-homme/pj-co...,Y2J6YwUDdw,ZmF6bQUAeQ,1,729,51310,NaN,NaN
3,IVANA DE BERTRANGE,4,4,FEMELLES,TROTTEUR FRANCAIS,PARTANT,4.0,SANS_OEILLERES,Patrice GENTIL,B. MARIE,...,1,2100,https://www.letrot.com/stats/fiche-homme/b-mar...,YmFabAUFYA,YmFabAUFYA,2,730,54310,NaN,NaN
4,INFINITY JET,5,4,FEMELLES,TROTTEUR FRANCAIS,PARTANT,5.0,SANS_OEILLERES,Mme Sylvie SAINT GEORGES,L. DURANTET,...,1,2100,https://www.letrot.com/stats/fiche-homme/l-dur...,ZmB7YQECZg,ZmB7YQECZg,2,730,52030,NaN,NaN
5,IDYLLE DE MAHEY,6,4,FEMELLES,TROTTEUR FRANCAIS,PARTANT,6.0,SANS_OEILLERES,M. SOULAS,CH. MOTTIER,...,1,2100,https://www.letrot.com/stats/fiche-homme/c-mot...,Z2Z6ZwECfA,YGJaYwUFaA,2,747,49050,NaN,NaN
6,ISSIA DE CHARLY,7,4,FEMELLES,TROTTEUR FRANCAIS,PARTANT,7.0,SANS_OEILLERES,TH. BLOQUET,P. GODEY,...,1,2100,https://www.letrot.com/stats/fiche-homme/p-god...,YmZabQUFfw,YGNbZAIAeQ,3,728,48230,NaN,NaN
7,INAYA MATIDY,8,4,FEMELLES,TROTTEUR FRANCAIS,PARTANT,8.0,SANS_OEILLERES,Cédric POSTOLLEC,T. LE BELLER,...,1,2100,https://www.letrot.com/stats/fiche-homme/t-le-...,YmJaZQoDcA,YWd4YgMGZA,1,732,50260,NaN,NaN
8,IRINUSHKA,9,4,FEMELLES,TROTTEUR FRANCAIS,PARTANT,9.0,SANS_OEILLERES,Mme M. STIHL,F. SENET,...,1,2100,https://www.letrot.com/stats/fiche-homme/f-sen...,bGV5ZwMBfw,bGV5ZwMBfw,2,734,53430,NaN,NaN
9,IRMA DU VIVIER,10,4,FEMELLES,TROTTEUR FRANCAIS,PARTANT,10.0,SANS_OEILLERES,Ecurie MINIER,B. ROBIN,...,1,2100,https://www.letrot.com/stats/fiche-homme/b-rob...,YGJaZwQFZA,YGJaZwQFZA,2,733,47970,NaN,NaN


In [536]:
for i in df.dtypes.items():
    t = str(i[1]).replace("object", "TEXT").replace("int64", "INT").replace("float64", "FLOAT").replace("bool", "BOOLEAN")
    print(i[0],"=", f"Column({t})")

nom = Column(TEXT)
numPmu = Column(INT)
age = Column(INT)
sexe = Column(TEXT)
race = Column(TEXT)
statut = Column(TEXT)
placeCorde = Column(FLOAT)
oeilleres = Column(TEXT)
proprietaire = Column(TEXT)
entraineur = Column(TEXT)
driver = Column(TEXT)
driverChange = Column(BOOLEAN)
indicateurInedit = Column(BOOLEAN)
musique = Column(TEXT)
nombreCourses = Column(INT)
nombreVictoires = Column(INT)
nombrePlaces = Column(INT)
nomPere = Column(TEXT)
nomMere = Column(TEXT)
ordreArrivee = Column(FLOAT)
jumentPleine = Column(BOOLEAN)
engagement = Column(BOOLEAN)
supplement = Column(INT)
handicapDistance = Column(INT)
poidsConditionMonteChange = Column(BOOLEAN)
tempsObtenu = Column(FLOAT)
reductionKilometrique = Column(FLOAT)
allure = Column(TEXT)
robe_code = Column(TEXT)
robe_libelleCourt = Column(TEXT)
robe_libelleLong = Column(TEXT)
dernierRapportDirect_typePari = Column(TEXT)
dernierRapportDirect_rapport = Column(FLOAT)
dernierRapportDirect_typeRapport = Column(TEXT)
dernierRapportDirect_indica

_GatheringFuture exception was never retrieved
future: <_GatheringFuture finished exception=TypeError("'NoneType' object is not iterable")>
Traceback (most recent call last):
  File "C:\Users\raves\anaconda3\lib\asyncio\tasks.py", line 280, in __step
    result = coro.send(None)
  File "<ipython-input-504-2bfb74fc93d6>", line 152, in _info_tableau_partant
    chevaux.extend(tableau_pmu)
TypeError: 'NoneType' object is not iterable


In [37]:
df = pd.read_csv("export-mises.csv", sep=";")
#df = df.iloc[75:]
df = df.loc[df["Pari"] == "ZE 5"]
df

,Date,Hippodrome,Course,Pari,Combinaison,Nombre de combinaisons,Mise,Statut,Gains,Dont bonus
6,2022-12-18,VINCENNES,9.0,ZE 5,9-10-11-13-14,1,"0,50 €",Payée,NaN,NaN
12,2022-12-18,VINCENNES,8.0,ZE 5,3-5-6-7-12,1,"0,50 €",Payée,NaN,NaN
13,2022-12-18,VINCENNES,8.0,ZE 5,3-5-6-7-12,1,"0,50 €",Annulée,NaN,NaN
23,2022-12-18,VINCENNES,6.0,ZE 5,3-5-6-8-10,1,"0,50 €",Payée,NaN,NaN
68,2022-12-21,VINCENNES,7.0,ZE 5,9-12-14-15-16,1,"0,50 €",Payée,NaN,NaN
83,2022-12-21,CAGNES SUR MER,5.0,ZE 5,2-8-11-14-15,1,"0,50 €",Payée,NaN,NaN
99,2022-12-22,CABOURG,3.0,ZE 5,1-2-4-9-11,1,"0,50 €",Payée,NaN,NaN
104,2022-12-22,MESLAY DU MAINE,8.0,ZE 5,2-3-5-7-8,1,"0,50 €",Payée,"5,55 €",NaN
113,2022-12-23,VINCENNES,7.0,ZE 5,2-4-6-8-13,1,"0,50 €",Payée,NaN,NaN
118,2022-12-23,VINCENNES,5.0,ZE 5,3-5-6-8-11,1,"0,50 €",Payée,NaN,NaN


In [38]:
df["Mise"] = df["Mise"].str.replace(" €", "").str.replace(",", ".").astype(float)
df["Gains"] = df["Gains"].str.replace(" €", "").str.replace(",", ".").astype(float)

In [39]:
df["Mise"].sum()

5.5

In [40]:
df["Gains"].sum()

5.55